In [1]:
import unittest
from typing import List
import numpy as np



In [2]:
from openptv_python.tracking_frame_buf import n_tupel
from openptv_python.constants import TR_UNUSED

In [3]:
def run_test(tcls):
    """
    Runs unit tests from a test class
    :param tcls: A class, derived from unittest.TestCase
    """
    suite = unittest.TestLoader().loadTestsFromTestCase(tcls)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [4]:

def take_best_candidates(src: np.ndarray, dst: np.ndarray, num_cams: int, num_cands: int, tusage: List[List[int]]) -> int:
    taken: int = 0

    # Sort candidates by match quality (.corr)
    src.sort(order = 'corr')

    # Take quadruplets from the top to the bottom of the sorted list
    # only if none of the points has already been used
    for cand in range(num_cands):
        has_used_target: bool = False
        for cam in range(num_cams):
            tnum: int = src[cand]['p'][cam]

            # If any correspondence in this camera, check that target is free
            if tnum > TR_UNUSED and tusage[cam][tnum] > 0:
                has_used_target = True
                break

        if has_used_target:
            continue

        # Only now can we commit to marking used targets.
        for cam in range(num_cams):
            tnum = src[cand]['p'][cam]
            if tnum > TR_UNUSED:
                tusage[cam][tnum] += 1
        dst[taken] = src[cand]
        taken += 1

    return taken



In [5]:
@run_test
class TestTakeBestCandidates(unittest.TestCase):
    def test_quadruplets(self):
        src = np.array([
            ([0, 1, 2, 3], 0.9),
            ([1, 2, 3, 4], 0.8),
            ([2, 3, 4, 5], 0.7),
            ([3, 4, 5, 6], 0.6)
        ], dtype=n_tupel.dtype)
        
        dst = [None] * len(src)
        num_cams = 4
        num_cands = len(src)
        tusage = [[0] * 10 for _ in range(num_cams)]

        taken = take_best_candidates(src, dst, num_cams, num_cands, tusage)

        self.assertEqual(taken, len(src))
        self.assertEqual(dst, src)

    def test_triplets(self):
        src = np.array([
            ([0, 1, 2, -1], 0.9),
            ([1, 2, 3, -1], 0.8),
            ([2, 3, 4, -1], 0.7),
            ([3, 4, 5, -1], 0.6)
        ], dtype=n_tupel.dtype)
        dst = [None] * len(src)
        num_cams = 4
        num_cands = len(src)
        tusage = [[0] * 10 for _ in range(num_cams)]

        taken = take_best_candidates(src, dst, num_cams, num_cands, tusage)

        self.assertEqual(taken, len(src))
        self.assertEqual(dst[:taken], src[:taken])

    def test_pairs(self):
        src = np.array([
            ([0, 1, -1, -1], 0.9),
            ([1, 2, -1, -1], 0.8),
            ([2, 3, -1, -1], 0.7),
            ([3, 4, -1, -1], 0.6)
        ], dtype = n_tupel.dtype)
        
        dst = np.tile(n_tupel, len(src))
        num_cams = 4
        num_cands = len(src)
        tusage = [[0] * 10 for _ in range(num_cams)]

        taken = take_best_candidates(src, dst, num_cams, num_cands, tusage)

        self.assertEqual(taken, len(src))
        print(dst[:taken])
        print(src[:taken])
        self.assertEqual(dst[:taken], src[:taken])

    def test_no_candidates(self):
        src = []
        dst = [None] * 10  # Just to test that dst is not modified
        num_cams = 4
        num_cands = 0
        tusage = [[0] * 10 for _ in range(num_cams)]

        taken = take_best_candidates(src, dst, num_cams, num_cands, tusage)

        self.assertEqual(taken, 0)
        self.assertEqual(dst, [None] * 10)


# if __name__ == '__main__':
# 


test_no_candidates (__main__.TestTakeBestCandidates.test_no_candidates) ... ERROR
test_pairs (__main__.TestTakeBestCandidates.test_pairs) ... FAIL
test_quadruplets (__main__.TestTakeBestCandidates.test_quadruplets) ... ERROR
test_triplets (__main__.TestTakeBestCandidates.test_triplets) ... FAIL

ERROR: test_no_candidates (__main__.TestTakeBestCandidates.test_no_candidates)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_580508/1012884397.py", line 65, in test_no_candidates
    taken = take_best_candidates(src, dst, num_cams, num_cands, tusage)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_580508/842572579.py", line 5, in take_best_candidates
    src.sort(order = 'corr')
TypeError: 'order' is an invalid keyword argument for sort()

ERROR: test_quadruplets (__main__.TestTakeBestCandidates.test_quadruplets)
------------------------------------------------------